<a href="https://colab.research.google.com/github/nendter/A-sl-bc/blob/main/notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Load dataset into environment

!pip install kiggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download grassknoted/asl-alphabet
!unzip -o asl-alphabet.zip -d /content/asl-ds/raw

In [2]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf

In [5]:
### Load dataset

def load_dataset(raw_folder_path, img_target_size=(128, 128), img_mode="L"):

  # Train
  train_raw_folder_path = os.path.join(raw_folder_path, "asl_alphabet_train/asl_alphabet_train/")
  train_imgs = []
  train_lbls = []

  for i, lbl in enumerate(os.listdir(train_raw_folder_path)):
    lbl_train_raw_folder_path = os.path.join(train_raw_folder_path, lbl)

    for j, img_name in enumerate(os.listdir(lbl_train_raw_folder_path)):
      img_train_raw_folder_path = os.path.join(lbl_train_raw_folder_path, img_name)
      with Image.open(img_train_raw_folder_path).convert(img_mode).resize(img_target_size) as img_data:
        img = np.array(img_data) / 255
        train_imgs.append(img)
        train_lbls.append(lbl)

  return np.array(train_imgs), np.array(train_lbls)

train_imgs, train_lbls = load_dataset("/content/asl-ds/raw")

In [ ]:
### Build Model

def build_model():
  input = tf.keras.Input(shape=(train_imgs.shape[0], train_imgs.shape[1], 1))
  conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), padding='SAME', activation='relu')(input)
  pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(conv1)

  conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=(5, 5), padding='SAME', activation='relu')(pool1)
  pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(conv2)

  flat = tf.keras.layers.Flatten()(pool2)
  fc1 = tf.keras.layers.Dense(units=1024, activation='relu')(flat)
  logits = tf.keras.layers.Dense(units=train_lbls.shape[0])(fc1)

  return logits

model = build_model()